<a href="https://colab.research.google.com/github/VabiDevelops/ds_Vaibhav_Vikram_Singh/blob/main/notebook_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ======================
# 1. Import Libraries
# ======================
import pandas as pd
import numpy as np

# ======================
# 2. Load the Dataset
# ======================
# If file is on your local system, upload via Google Colab’s file uploader
from google.colab import files

uploaded = files.upload()   # select "historical_data.csv" when prompted

# Read into pandas DataFrame
hist = pd.read_csv("historical_data.csv")

# Quick look at the data
hist.head()


KeyboardInterrupt: 

In [ ]:
# ======================
# 3. Extract Table Info
# ======================
print("Shape of dataset:", hist.shape)    # rows x columns
print("\nColumn names:", hist.columns.tolist())
print("\nMissing values per column:\n", hist.isnull().sum())


In [ ]:
# ======================
# 4. Handle Timestamp Conversion
# ======================
# Convert 'Timestamp IST' into datetime format
hist['Timestamp IST'] = pd.to_datetime(hist['Timestamp IST'], errors='coerce')

# Extract date and time into separate columns
hist['trade_date'] = hist['Timestamp IST'].dt.date
hist['trade_time'] = hist['Timestamp IST'].dt.time

# Also, if 'Timestamp' column is in milliseconds since epoch, convert it
if 'Timestamp' in hist.columns:
    hist['Timestamp_utc'] = pd.to_datetime(hist['Timestamp'], unit='ms', errors='coerce')


In [ ]:
# ======================
# 5. Feature Engineering
# ======================

# is_win: 1 if trade profitable, else 0
hist['is_win'] = np.where(hist['closedPnL'] > 0, 1, 0)

# pnl_normalized: profit relative to trade size
hist['pnl_normalized'] = np.where(hist['Size USD'] != 0,
                                  hist['closedPnL'] / hist['Size USD'],
                                  np.nan)

# fee_adjusted_pnl: closedPnL minus fees
if 'Fee' in hist.columns:
    hist['fee_adjusted_pnl'] = hist['closedPnL'] - hist['Fee']
else:
    hist['fee_adjusted_pnl'] = hist['closedPnL']   # fallback if no Fee column


In [ ]:
# ======================
# 6. Identify Suspicious Zero PnL Trades
# ======================
zero_pnl_dummy_trades = hist[(hist['closedPnL'] == 0) &
                             (hist['Transaction Hash'].astype(str).str.startswith('0x000'))]

print("Rows with closedPnL=0 and dummy Transaction Hash:")
print(zero_pnl_dummy_trades.shape)
zero_pnl_dummy_trades.head()
